<a href="https://colab.research.google.com/github/MoChiaoKai/Kai0214/blob/main/0504_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyngrok flask flask_ngrok # for colab
!pip install -q flask line-bot-sdk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 11.2 MB/s eta 0:00:00


In [ ]:
# colab

from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get("NGROK_AUTH_TOKEN"))
    ngrok.connect(5000)
    run_with_ngrok(app)

# 啟動ngrok tunnel
from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
     Configuration, ApiClient, MessagingApi,
     ReplyMessageRequest,
     TextMessage,
     TemplateMessage, ConfirmTemplate, MessageAction,
     ImageCarouselTemplate,
     ImageCarouselColumn,
     PostbackAction,
     URIAction
)
app = Flask(__name__)

# 設定Handler
configuration = Configuration(access_token=userdata.get("LINE_CHANNEL_ACCESS_TOKEN"))
handler = WebhookHandler(userdata.get("LINE_CHANNEL_SECRET"))

@app.route("/callback", methods=["POST"])
def callback():
    signature = request.headers["X-Line-Signature"]
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return "OK"

@handler.add(MessageEvent, message=TextMessageContent)
def message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == "confirm":
            confirm_template = ConfirmTemplate(
                text="你確定要學習嗎？",
                actions=[
                    MessageAction(label="是的", text="我想要學習"),
                    MessageAction(label="不是", text="我只是路過的啦")
                ]
            )
            reply = TemplateMessage(
                alt_text="這是確認模板",
                template=confirm_template
            )
        elif action == "carousel":
            carousel_template = CarouselTemplate(
                columns=[
                    CarouselColumn(
                        thumbnail_image_url="https://upload.wikimedia.org/wikipedia/zh/7/7d/Summer_Wars_poster.jpg",
                        title="夏日大作戰",
                        text="虛擬世界的危機！",
                        actions=[
                            MessageAction(label="影片介紹", text="https://zh.wikipedia.org/zh-tw/%E5%A4%8F%E6%97%A5%E5%A4%A7%E4%BD%9C%E6%88%B0"),
                            MessageAction(label="想看", text="我想再看一遍")
                        ]
                    ),
                    CarouselColumn(
                        thumbnail_image_url="https://upload.wikimedia.org/wikipedia/zh/thumb/4/4f/Castle_of_Cagliostro_poster.png/250px-Castle_of_Cagliostro_poster.png",
                        title="魯邦三世：卡里奧斯特羅城",
                        text="義大利古堡奇遇記",
                        actions=[
                            MessageAction(label="影片介紹", text="https://zh.wikipedia.org/zh-tw/%E9%AD%AF%E9%82%A6%E4%B8%89%E4%B8%96%EF%BC%9A%E5%8D%A1%E9%87%8C%E5%A5%A7%E6%96%AF%E7%89%B9%E7%BE%85%E5%9F%8E"),
                            MessageAction(label="想看", text="我想把這三年卡通補回來")
                        ]
                    )
                ]
            )
            reply = TemplateMessage(
                alt_text="這是轉盤模板",
                template=carousel_template
            )
        else:
            reply = TextMessage(text="Thanks!")

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[reply]
            )
        )


ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://c62e-34-106-192-230.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [03/Jun/2025 06:06:40] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jun/2025 06:06:49] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jun/2025 06:09:08] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jun/2025 06:09:11] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jun/2025 06:09:20] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jun/2025 06:09:22] "POST /callback HTTP/1.1" 200 -
ERROR:__main__:Exception on /callback [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 917,